# Getting Start

## Install JanusQ

JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended, as all functions have been tested on it. Linux (Ubuntu 22.04 latest) platform and Python (3.10) is prefered when instaling by wheel or soruce code.

**From docker (Recommended)**

Pull docker image

```bash
  docker pull janusq/janusq:latest
```

Run docker image

```bash
  docker run -itd -p 8888:22 -p 9999:23 --name tutorial janusq/janusq
```

The docker can be accessed via

```bash
  ssh root@localhost -p 8888
```

or

```bash
  docker exec -it tutorial bash
```

The source code is in "/JanusQ/janusq" and examples can be directly run in "/JanusQ/examples/ipynb". The jupyter notebook can be visited in http://localhost:9999/lab.

**From wheel**

Download [janusq.whl](https://github.com/JanusQ/JanusQ/blob/main/dist/janusq-0.1.0-py3-none-any.whl) and install with `pip`

```bash
  pip install janusq.whl
```  

**From source code**

Pull the source code from github and install the dependencies

```bash
  git clone git@github.com:JanusQ/JanusQ.git
  cd JanusQ
  pip install -r requirements.txt
```  

Set up for HyQSAT

```bash
  cd ./hyqsat
  cmake .
  make install
  cp libm* ../janusq/hyqsat
  cp minisat_core ../janusq/hyqsat
```  

## Structure of JanusQ

- QuCT
  - Vectorization
    - "janusq/analysis/vectorization.py": Python script to generate the path table and vecterization of circuits, which serves as a upstream model for various analysis and optimization tasks.
  - Fidelity preidiction and optimization
    - "janusq/analysis/fidelity_prediction.py": Python script to train a downstream model for circuit fidelity prediction.
    - "janusq/optimizations/mapping/mapping_ct.py"：Python script of a typical compilation flow, including routing and scheduling. The compilation flow transforms the circuit to satisfy the processor topology so as to optimize the circuit fidelity based on the prediction model.
  - Unitary decomposition
    - "janusq/analysis/unitary_decompostion.py": Python script that takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.
- MorphQPV
- QuFEM
  - "janusq/optimizations/readout_mitigation/fem/benchmarking.py": Python script to generate a circuit for measuring calibration matrices.
  - "janusq/optimizations/readout_mitigation/fem/mitigation.py": Python script to implement iterative calibration, which includes quantifying interactions between qubits and constructing Bayesian networks.
  - "janusq/optimizations/readout_mitigation/fem/tools.py": Python script to implement data format conversion.
- HyQSAT
  - "janusq/hyqsat/common": Directory holding common functions about cnf files, such as "read_cnf.py".
  - "janusq/hyqsat/solver.py": Python script holding different APIs used to solve sat problem.
- Choco-Q

## Q&A

1. Error "Permission denied 'minisat_core'"

    Run following command

    ```bash
      chmod +x janusq/hyqsat/minisat_core
    ```

2. Error "Library not loaded: @rpath/libpython3.10.dylib" 

    Run following command

    ```bash
      install_name_tool -add_rpath /path/to/python3.10/lib janusq/hyqsat/minisat_core
    ```

## Test JanusQ

In [1]:
import os
os.chdir("..")
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

### Test QuCT

In [2]:
# test quct
from janusq.data_objects.backend import LinearBackend
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel

# define a 10-qubit backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

up_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
up_model.train(circuits, multi_process=False, remove_redundancy = False)

100%|██████████| 300/300 [00:00<00:00, 380.62it/s]


### Test QuFEM

In [3]:
# test readout calibration
from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.data_objects.backend import LinearBackend
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.optimizations.readout_mitigation.fem import  EnumeratedProtocol
from janusq.data_objects.algorithms import get_algorithm_circuits

n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algorithm_circuits(n_qubits, backend, algs = ['ghz'])[0]

samples = 10000

simulator = NoisySimulator(
    backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)


protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()

all_statuscnts = [
    simulator.execute(cir_protocol, samples)
    for cir_protocol in circuits_protocol
]

print(all_statuscnts)


/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


[{'011': 5, '100': 41, '001': 335, '101': 1, '010': 133, '000': 9485}, {'000': 974, '010': 12, '101': 39, '011': 115, '100': 4, '001': 8856}, {'011': 6, '110': 2, '100': 44, '001': 296, '000': 9514, '010': 136, '101': 2}, {'011': 290, '111': 1, '100': 1, '001': 24, '110': 32, '000': 615, '010': 9037}, {'101': 5, '000': 55, '010': 893, '111': 30, '011': 8445, '110': 2, '001': 570}, {'100': 4, '001': 19, '110': 30, '010': 9033, '000': 616, '111': 1, '011': 297}, {'101': 1, '110': 1, '100': 35, '001': 317, '011': 6, '010': 131, '000': 9509}, {'001': 8869, '100': 6, '111': 2, '011': 120, '101': 40, '010': 12, '000': 951}, {'101': 1, '100': 35, '001': 279, '011': 5, '010': 135, '000': 9545}, {'111': 4, '100': 9462, '001': 2, '110': 142, '010': 3, '000': 86, '101': 301}, {'110': 11, '001': 77, '100': 969, '111': 114, '000': 10, '101': 8819}, {'111': 1, '001': 2, '110': 149, '100': 9474, '010': 3, '000': 67, '101': 304}, {'101': 24, '011': 2, '111': 286, '110': 8992, '100': 612, '000': 4, '01

### Test HyQSAT

In [4]:
# test hyqsat calibration
from janusq.hyqsat import readCNF

# input cnf flie
file_path = "cnf_examples/UF50/uf50-01.cnf"
print(readCNF(file_path))

([(-3, 7, 36), (-3, -42, -48), (-41, -47, -49), (8, 17, -40), (-21, -31, -39), (-22, 36, 49), (14, 27, 38), (6, 15, -18), (6, 7, -43), (-7, 23, 34), (2, -13, 14), (2, -42, 47), (3, -33, -35), (40, 44, 49), (31, 36, 50), (-3, -36, -37), (26, -29, 43), (15, 29, -45), (-11, 18, 24), (6, -26, -47), (-10, -33, -50), (6, 16, 32), (-34, 37, 41), (7, -17, -28), (19, -44, 46), (7, 22, -48), (3, 34, 39), (31, -43, 46), (23, -27, 32), (-18, 37, -50), (5, 11, 20), (6, -24, -45), (-14, -23, -34), (20, 21, -22), (-17, 24, 50), (-24, -25, -27), (3, 21, 35), (-26, -36, 47), (-28, -45, 49), (-6, 12, -21), (-15, -17, -39), (2, -14, 41), (-23, 25, 36), (-3, -39, -40), (20, 35, 50), (27, 31, -39), (-15, -40, 45), (34, 35, 50), (-1, 12, -48), (18, -30, -35), (-24, -25, 27), (-4, -12, -33), (-24, -37, -43), (31, -37, -44), (-9, 14, -38), (-16, 33, 34), (4, -5, -35), (-3, -19, -21), (-29, -35, -36), (7, 36, -43), (14, 30, 41), (-7, -24, -35), (6, 35, -42), (-1, -15, 39), (-16, 27, 49), (-10, -37, 49), (-3, -